# Seafloor Bacterial Floc Analysis Using Regions Information

This notebook shows an example of doing an analysis of water column "floc" using Pangeo and the regions data from Aaron's [CamHD_motion_metadata](https://github.com/CamHD-Analysis/CamHD_motion_metadata) repo. The goal of this work is to understand changes in the concentration of floc, which is bacterial material that has been flushed from the hydrothermal system into the ocean. Changes in floc are a potential indicator of changes in the hydrothermal system, possibly resulting from a magmatic event or seismic swarm.
 
 
In this notebook we analyze a large number of OOI HD video camera frames to establish a proxy for the floc concentration, and then display the results using a two-dimensional multivariate histogram.

#### Get a list of CamHD files to process

In [ ]:
import json
import pandas as pd
import pycamhd as camhd
import numpy as np
from dask import delayed
import dask.array as dsa

In [ ]:
url = []
filename = []
scene_tag = []
deployment = []
start_framed = []
end_framed = []

with open('region.txt') as f:
    for line in f:
        with open(line.strip()) as l:
            data = json.load(l)
            for region in data['regions']:
                if region['type'] == 'static':
                    try:
                        if '_p2_z0' in region['sceneTag']:
                            url.append('https://rawdata.oceanobservatories.org/files' + data['movie']['URL'])
                            filename.append((data['movie']['URL'].split('/')[-1]))
                            scene_tag.append(region['sceneTag'])
                            deployment.append(int(scene_tag[-1].split('_')[0][1]))                            
                            start_framed.append(int(region['startFrame']))
                            end_framed.append(int(region['endFrame']))
                    except:
                        continue

## To Whom It May Concern: 
#### Here is a cell that starts by defining a proper buffer for our start and end frames. The idea being that the buffer will provent us from pulling frames that may include the end or beggining of the CamHD's movement.
#### We go on to create a dataframe of relevent information from a list of movies that fit specifications made in the cell above
#### From here we apply a function to the rows of the dataframe that takes the defined frame_interval for the range give by the start_frame to end_frame 
#### We then add that the list of frames as a column 'window' to our data
#### We then go through our dataframe and filter it to display only the rows from the given deployment (5)

In [ ]:
frame_buffer = 10
frame_interval = 30
start_frame = [x+frame_buffer for x in start_framed]
end_frame = [x-frame_buffer for x in end_framed]
scene_windows = pd.DataFrame({'filename': filename, 'url': url, 'scene_tag': scene_tag, 'deployment': deployment, 'start_frame': start_frame, 'end_frame': end_frame})
def window(row):
    return range(row.start_frame, row.end_frame, frame_interval)

scene_windows['window'] = scene_windows.apply(window, axis=1)
scene_windows.loc[(scene_windows.deployment == 5)]

scene_windows.tail()


In [ ]:

# delayed_frame_list = []
# for i, filename in enumerate(scene_windows):
#     delayed_moov_atom = delayed(camhd.get_moov_atom)(url)
#     delayed_frame = delayed(camhd.get_frame)(filename, frame_numbers[i], 'gray16le', delayed_moov_atom)
#     for i, start_frame in enumerate(scene_windows.start_frame):
#         i == 1
#         print(delayed_frame_list)
#         break
delayed_frame_list = []
for i, filename in enumerate(scene_windows):
    delayed_moov_atom = delayed(camhd.get_moov_atom)(url)
    delayed_frame = delayed(camhd.get_frame)(url, window, 'gray16le', delayed_moov_atom)
    delayed_frame_list.append(dsa.from_delayed(delayed_frame, (1080, 1920), np.uint16))
    i == 1
    print(delayed_frame_list)
    break
   
# delayed_frame_array = dsa.stack(delayed_frame_list)
# delayed_frame_array

In [ ]:
delayed_frame_list

In [ ]:
delayed_frame_list = []
for i, filename in enumerate(scene_windows):
    delayed_file = filename
    delayed_frame_list.append(delayed_file)
    delayed_frame = window
    delayed_frame_list.append(delayed_frame)
    i == 1
    print(delayed_frame_list)

#### Define the frame numbers from each file to process

In [ ]:
print(delayed_frame_list)

In [ ]:
# interval = 30
# padding = 10
# # frame_windows = []
# d = {'start': scene_windows.start_frame+padding, 'end': scene_windows.end_frame-padding,'filenames': scene_windows.filename}
# # end = scene_windows.end_frame-padding
# # range(start, end, interval)
# frame_windows = pd.DataFrame(index=None, columns= ['start', 'end', 'filenames'], data=d, dtype=None, copy=False)

# # # frame_windows.append(window)
# def window(row):
#     return range(row.start, row.end, 30)

# frame_windows['window'] = frame_windows.apply(window, axis=1)

# # # DataFrame.append(other, ignore_index=False, verify_integrity=False, sort=None)
# # # frame_windows.append(list(start_frame+padding, scene_windows.end_frame-padding, interval))
# # # # frame_numbers.append(list(range(start_frame+padding, scene_windows.end_frame[i]-padding, interval)))

# # #     if i == 1:
# # #         break


In [ ]:
# for [first iterating variable] in [outer loop]: # Outer loop
#     [do something]  # Optional
#     for [second iterating variable] in [nested loop]:   # Nested loop
#         [do something]  
        
# delayed_frame_list = []
# for i, filename in enumerate(filenames):
#     delayed_file = filename
#     delayed_frame_list.append(delayed_file)
#     for j, frame_number in enumerate(frame_numbers):
#         delayed_frame = frame_number
#         delayed_frame_list.append(delayed_frame)
# delayed_file_info = []

# for i, filename in filename.index:
#     delayed_file = filename[i]
#     delayed_frame_list.append(delayed_file)
#     for i in frame_number.index:
#         delayed_frame = frame_number
#         delayed_frame_list.append(delayed_frame)
# delayed_frame_list
# delayed_file_info = []
delayed_frame_list = []
for i, filename in enumerate(filenames):
    delayed_file = filename
    delayed_frame_list.append(delayed_file)
    delayed_frame = frame_numbers[i]
    delayed_frame_list.append(delayed_frame)
    #for i, delayed_file in enumerate(delayed_frame_list):
     #   delayed_frame = frame_numbers[i]
      #  delayed_frame_list.append(delayed_frame)
#         i == (9)
#     break
#         print(delayed_frame_list)
#         break

https://docs.python.org/3.3/library/functions.html#zip
Make an iterator that aggregates elements from each of the iterables.

Returns an iterator of tuples, where the i-th tuple contains the i-th element from each of the argument sequences or iterables. 
The iterator stops when the shortest input iterable is exhausted. With a single iterable argument, it returns an iterator of 1-tuples. 
With no arguments, it returns an empty iterator.

These frame numbers correspond to times when the camera system is looking over the "shoulder" of Mushroom vent.

#### Set up a delayed Dask array of images

In [ ]:
import pycamhd as camhd
import numpy as np
from dask import delayed
import dask.array as dsa

In [ ]:
delayed_frame_list = []
for i, filename in enumerate(filenames):
    delayed_moov_atom = delayed(camhd.get_moov_atom)(filename)
    delayed_frame = delayed(camhd.get_frame)(filename, frame_numbers, 'gray16le', delayed_moov_atom)
    delayed_frame_list.append(dsa.from_delayed(delayed_frame, (1080, 1920), np.uint16))
   
delayed_frame_array = dsa.stack(delayed_frame_list)
delayed_frame_array

In [ ]:

delayed_frame_list = []
for i, filename in enumerate(filenames):
    delayed_moov_atom = delayed(camhd.get_moov_atom)(filename)
    delayed_frame = delayed(camhd.get_frame)(filename, frame_numbers[i], 'gray16le', delayed_moov_atom)
    for i, start_frame in enumerate(scene_windows.start_frame):
        i == 1
        print(delayed_frame_list)
        break

In [ ]:
df = pd.DataFrame(delayed_frame_list)

In [ ]:
next(delayed_moov_atom.dask.items())

In [ ]:
df.to_csv("doody.csv")

A dask array is in many ways like a numpy array, except in this case it holds a set of instructions for how to acquire each chunk of the array, which makes it easy to farm this array out to workers in the cloud using the [distributed](http://distributed.readthedocs.io/en/latest/#) scheduler.

#### Show one of the images

In [ ]:
frame = delayed_frame_array[0].compute()
frame.shape

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.rc('figure', figsize=(11, 11))
fig, ax = plt.subplots()
im1 = ax.imshow(frame)
im1.set_cmap('gray')
plt.yticks(np.arange(0,1081,270))
plt.xticks(np.arange(0,1921,480))
rect = patches.Rectangle((10,10),1024,1024,linewidth=1.5,edgecolor='w',facecolor='none')
ax.add_patch(rect)
plt.show();

#### Show the filter that will be used to filter images in the frequency domain
To deal with variations in lighting and high-frequency noise, we filter each subimage using a Butterworth bandpass filter.

In [ ]:
def butterworth(d1, d2, n):
    x = np.arange(-1024/2+0.5,1024/2+1-0.5)
    xx, yy = np.meshgrid(x, x)
    d = np.sqrt(xx**2+yy**2)
    bff = (1 - (1./(1 + (d/d1)**(2*n))))*(1/(1 + (d/d2)**(2*n)))
    return bff

In [ ]:
d1 = 20 # low cut wavenumber
d2 = 400 # high cut wavenumber
n = 4
bff = butterworth(d1, d2, n)
plt.rc('figure', figsize=(6, 6))
imgplot = plt.imshow(bff, cmap='gray')

#### Define the floc proxy function
The floc proxy is simply the number of pixels in each filtered subimage that have a value greater than 4000.

In [ ]:
def frame_filter(frame, d1, d2, n):
    if frame.ndim == 3 and frame.shape[0] == 1:
        I = np.squeeze(frame[0, 0:1024, 0:1024])
    else:
        I = frame[0:1024, 0:1024]
    bff = butterworth(d1, d2, n)
    I_fft = np.fft.fft2(I)
    I_fft_shift = np.fft.fftshift(I_fft)
    I_fft_shift_filt = I_fft_shift*bff # filter with the Butterworth filter
    I_fft_filt = np.fft.ifftshift(I_fft_shift_filt)
    I_filt = np.fft.ifft2(I_fft_filt)
    return I_filt

In [ ]:
def calc_floc_proxy(frame, d1, d2, n):
    I_filt = frame_filter(frame, d1, d2, n)
    return np.array([(np.absolute(I_filt)>4000).sum()])

#### Show example for one frame

In [ ]:
I_filt = frame_filter(frame, d1, d2, n)

In [ ]:
plt.rc('figure', figsize=(6, 6))
imgplot = plt.imshow(np.absolute(I_filt)>4000, cmap='gray')
plt.title('floc_proxy value = %i' % (np.absolute(I_filt)>4000).sum());

#### Assemble a new Dask array including our computation using map_blocks

In [ ]:
floc_proxy = dsa.map_blocks(calc_floc_proxy, delayed_frame_array, d1, d2, n, dtype='i8', drop_axis=[1,2])
floc_proxy

#### Compute the floc_proxy (subset)

In [ ]:
%%time
results = floc_proxy[0:40].compute()

#### Start a Dask cluster

In [ ]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=30)
cluster

In [ ]:
from dask.distributed import Client
client = Client(cluster)
client

In [ ]:
%%time
results = floc_proxy[0:400].compute()

#### Calculate all the results

In [ ]:
print('Number of images: %i' % len(floc_proxy))
print('Size of dataset (GB): %i' % round(len(floc_proxy)*1080*1920*2/1024/1024/1024))

In [ ]:
%%time
results = floc_proxy.compute()

#### Get a timestamp for each frame

In [ ]:
dbcamhd = pd.read_json('/home/jovyan/rte-camhd/tim/dbcamhd.json', orient="records", lines=True)

In [ ]:
import datetime, math
import matplotlib.dates as dates
frame_timestamp = []
for i, filename in enumerate(filenames):    
    timestamp = dbcamhd['timestamp'][dbcamhd.filename == filename].iloc[0]
    timestamp = timestamp + frame_numbers[i]/29.97
    dt = datetime.datetime.fromtimestamp(timestamp)
    frame_timestamp.append(dates.date2num(dt))

#### Plot a two-dimensional multivariate histogram of the results

In [ ]:
plt.rc('font', size=11)
fig, ax = plt.subplots()
fig.set_size_inches(14, 6)
fig.frameon = False
hb1 = ax.hexbin(frame_timestamp, results, vmin=0.1, vmax=2, bins='log', linewidths=0.25,
  gridsize=(225,4500), mincnt=1, cmap=plt.cm.BuPu)
fig.colorbar(hb1)
ax.set_ylim([0, 8000])
ax.set_xlim([frame_timestamp[0],frame_timestamp[-1]])
ax.yaxis.grid(True)
ax.xaxis.grid(True)
months = dates.MonthLocator()  # every month
monthsFmt = dates.DateFormatter('%b %Y')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
plt.ylabel('Floc Proxy Value');

Starting in mid-June a large "floc event" occurs where the floc proxy values increase on average by about a factor of ten. The cause of this floc event is being investigated.

### References

 - [Pangeo](http://pangeo-data.org/)
 - [PyCamHD](https://github.com/tjcrone/pycamhd)
 - [CamHD Raw Data Archive](https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301)
 - [AGU Abstract](https://agu.confex.com/agu/fm16/meetingapp.cgi/Paper/192670)
 - [AGU Poster](https://drive.google.com/open?id=0B-dWW4GM434obGpTM0FZME10Nkk)
 - [Dask](http://dask.pydata.org/en/latest/)